# StudyFlow Assistant (Colab Runner)

In [ ]:
#@title 1. Upload & Extract Project
#@markdown Click the button that appears below to upload studyflow_assistant.zip from your Chromebook's Downloads.

from google.colab import files
import os, shutil

print('Click the Choose Files button below and select studyflow_assistant.zip')
uploaded = files.upload()

PROJECT = '/content/studyflow_assistant'
if os.path.exists(PROJECT):
    shutil.rmtree(PROJECT)

zip_name = list(uploaded.keys())[0]
!unzip -qo "{zip_name}" -d /content/studyflow_assistant
print(f'Extracted! Files: {os.listdir(PROJECT)}')

In [ ]:
#@title 2. Install Dependencies
!pip install -q playwright==1.52.0 apscheduler==3.11.0 python-dotenv==1.1.0 tenacity==9.0.0 pydantic==2.11.3 pydantic-settings==2.9.1
!playwright install chromium
!playwright install-deps
print('All dependencies installed!')

In [ ]:
#@title 3. Set Up Environment (.env)
#@markdown Enter your Gemini API key below:
GEMINI_API_KEY = '' #@param {type:"string"}

env_path = '/content/studyflow_assistant/.env'
with open(env_path, 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
print('.env written')

if not GEMINI_API_KEY:
    print('WARNING: No API key set. Drafts will fail.')

In [ ]:
#@title 4. First-Time Login (opens browser you can see)
#@markdown A clickable link appears below -- open it to see the browser.
#@markdown Sign in to Google Classroom, then come back and STOP this cell.

!apt-get update -qq > /dev/null 2>&1
!apt-get install -y -qq xvfb x11vnc novnc websockify > /dev/null 2>&1

import subprocess, os, time

!pkill -f Xvfb 2>/dev/null; pkill -f x11vnc 2>/dev/null; pkill -f websockify 2>/dev/null
time.sleep(1)

subprocess.Popen(['Xvfb', ':99', '-screen', '0', '1280x720x24'])
time.sleep(1)
os.environ['DISPLAY'] = ':99'

subprocess.Popen(
    ['x11vnc', '-display', ':99', '-forever', '-nopw', '-shared', '-rfbport', '5900'],
    stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
)
time.sleep(1)

subprocess.Popen(
    ['websockify', '--web', '/usr/share/novnc', '6080', 'localhost:5900'],
    stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
)
time.sleep(2)

from google.colab.output import eval_js
from IPython.display import HTML, display

novnc_url = eval_js("google.colab.kernel.proxyPort(6080)")
full_url = f"{novnc_url}vnc.html?autoconnect=true&resize=remote"

display(HTML(f'''
<h3>Click this link to see the browser:</h3>
<a href="{full_url}" target="_blank" style="font-size:20px; color:blue;">
    >>> Open Browser View (noVNC) <<<
</a>
<p style="color:red; font-weight:bold;">After signing in to Classroom, come back here and click STOP.</p>
'''))

import asyncio, sys
os.chdir('/content/studyflow_assistant')
sys.path.insert(0, '/content/studyflow_assistant')

async def manual_login():
    from playwright.async_api import async_playwright
    pw = await async_playwright().start()
    ctx = await pw.chromium.launch_persistent_context(
        user_data_dir='/content/studyflow_assistant/.browser_data',
        headless=False,
        args=['--no-sandbox', '--disable-dev-shm-usage', '--disable-gpu'],
        viewport={'width': 1280, 'height': 720},
    )
    page = ctx.pages[0] if ctx.pages else await ctx.new_page()
    await page.goto('https://accounts.google.com/ServiceLogin')
    print('Browser opened -- go to the noVNC tab and sign in.')
    while True:
        await asyncio.sleep(5)

try:
    await manual_login()
except (KeyboardInterrupt, asyncio.CancelledError):
    print('Login session saved!')

In [ ]:
#@title 5. Run StudyFlow
#@markdown Scans assignments, generates drafts, pastes into Google Docs.

import subprocess, os
os.chdir('/content/studyflow_assistant')
if 'DISPLAY' not in os.environ:
    os.environ['DISPLAY'] = ':99'

result = subprocess.run(
    ['python', 'main.py', 'run'],
    capture_output=False,
    text=True,
    cwd='/content/studyflow_assistant',
    env={**os.environ, 'STUDYFLOW_MODE': 'run'},
)
print(f'\nExit code: {result.returncode}')